# 1. N-Queens backtracking - Naive

- Place Queen by columns
<img src="./img/1.jpg" width="300"/>

- Check feasible O(N)
<img src="./img/2.jpg" width="600"/>

In [1]:
class N_Queen(object):
    def __init__(self):
        self._N = None
        self._board = None 

    def __print_ans(self):
        for i in range(self._N):
            for j in range(self._N):
                end_c = '\n' if j == self._N - 1 else ' '
                print(self._board[i][j], end=end_c)

    def __check(self, r, c):
        # Check row O(N)
        for j in range(c):
            if self._board[r][j] == 1:
                return False
        
        # Check upper diag(\) O(n)
        i, j = r, c
        while i>=0 and j>=0:
            if self._board[i][j] == 1:
                return False
            i, j = i-1, j-1
        
        # Check lower diag(/) O(n)
        i, j = r, c
        while i < self._N and j>=0:
            if self._board[i][j] == 1:
                return False
            i, j = i+1, j-1
        
        # All constraints feasible
        return True

    def __dfs(self, c):
        # Solution found
        if c == self._N:
            self.__print_ans()
            return True

        # For col c, try placing row by row
        for r in range(self._N):
            if self.__check(r, c):
                # Place
                self._board[r][c] = 1

                # Move to next col
                if self.__dfs(c+1): return True

                # If c+1 failed, backtrack
                self._board[r][c] = 0
        
        # All attemps failed, No solution
        return False

    def solve(self, N=4):
        # Reset board
        self._N = N
        self._board = [ [0] * N for _ in range(N)]

        # Solve
        if self.__dfs(0) == False:
            print('No solution')

In [2]:
%%time
bactrack = N_Queen()

bactrack.solve(N=20)

1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
CPU times: user 3.94 s, sys: 0 ns, total: 3.94 s
Wall time: 3.94 s


# 2. N-Queens - Constraint Programming
- Place Queen by columns
<img src="./img/1.jpg" width="450"/>

- Define constraints, check in O(1)

| <img src="./img/3.jpg" width="500"/> | <img src="./img/4.jpg" width="650"/> |
|--------------------------------------|--------------------------------------|

In [3]:
class N_Queen_CP(object):
    def __init__(self):
        self._N = None
        self._board = None

        self._row_constr = None
        self._slash_constr = None
        self._backslash_constr = None

    def __print_ans(self):
        for i in range(self._N):
            for j in range(self._N):
                end_c = '\n' if j == self._N - 1 else ' '
                print(self._board[i][j], end=end_c)

    def __check(self, r, c):
        # Check row, slash, backslash constraints O(1)
        if self._row_constr[r] or \
            self._slash_constr[r+c] or \
            self._backslash_constr[r - c + self._N - 1]: return False

        # All constraints feasible
        return True

    def __dfs(self, c):
        # Solution found
        if c == self._N:
            self.__print_ans()
            return True

        # For col c, try placing row by row
        for r in range(self._N):
            if self.__check(r, c):
                # Place and Update constraints
                self._board[r][c] = 1
                self._row_constr[r] = True
                self._slash_constr[r+c] = True
                self._backslash_constr[r-c + self._N - 1] = True

                # Move to next col
                if self.__dfs(c+1): return True

                # If c+1 failed, backtrack and Remove constraints
                self._board[r][c] = 0
                self._row_constr[r] = False
                self._slash_constr[r+c] = False
                self._backslash_constr[r-c + self._N - 1] = False
        
        # All attemps failed, No solution
        return False

    def solve(self, N=4):
        # Reset board
        self._N = N
        self._board = [ [0] * N for _ in range(N)]

        self._row_constr = [False] * N
        self._slash_constr = [False] * (2*N-1)
        self._backslash_constr = [False] * (2*N-1)

        # Solve
        if self.__dfs(0) == False:
            print('No solution')

In [4]:
%%time
CP = N_Queen_CP()

CP.solve(N=20)

1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
CPU times: user 587 ms, sys: 3.9 ms, total: 591 ms
Wall time: 588 ms
